# Exploratory Data Analysis (EDA) - Candidates Dataset

## Workshop: ETL Pipeline for Recruitment Data

This notebook performs an initial exploration of the raw dataset (`candidates.csv`) to understand its structure, quality, and key characteristics **before** starting the ETL process.

### Why do we do EDA?
- Understand the shape and types of our data
- Detect null values, duplicates, or inconsistencies
- Explore distributions and patterns
- Make informed decisions for the Transform phase

## 1. Setup & Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plot style
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)

# Load the CSV file
# IMPORTANT: The separator is semicolon (;), not comma (,)
df = pd.read_csv('../data/raw/candidates.csv', sep=';')

print(f'Dataset shape: {df.shape[0]} rows x {df.shape[1]} columns')
df.head()

## 2. Data Types & Structure

Let's check what type of data each column holds. This is important because:
- Dates stored as strings need to be converted
- Numeric columns should be int/float, not text
- This helps us plan our transformations

In [ ]:
df.dtypes

In [ ]:
df.info()

### Key observations:
- `Application Date` is stored as **string** → needs conversion to datetime
- `YOE`, `Code Challenge Score`, `Technical Interview Score` are correctly **int64**
- Text columns are **object/string** type — expected

## 3. Null Values Check

Null values can break our ETL pipeline or produce incorrect KPIs. Let's check if any exist.

In [ ]:
null_summary = df.isnull().sum()
print(f'Total null values in dataset: {df.isnull().sum().sum()}')
print('\nNull count per column:')
null_summary

**Result:** No null values found. The dataset is clean in this regard.

## 4. Duplicate Analysis

Duplicates could inflate our metrics. Let's check for:
- Fully duplicated rows
- Duplicated emails (which should be unique per candidate)

In [ ]:
print(f'Fully duplicated rows: {df.duplicated().sum()}')
print(f'Duplicated emails: {df["Email"].duplicated().sum()}')

# Show some duplicate emails
if df['Email'].duplicated().sum() > 0:
    dup_emails = df[df['Email'].duplicated(keep=False)].sort_values('Email')
    print(f'\nSample of duplicated emails:')
    print(dup_emails.head(10).to_string())

### Observation:
- **0 fully duplicated rows** — good
- **167 duplicated emails** — this means some candidates applied more than once
- **Decision:** We will keep all rows since the grain of our fact table is *one row per application*, not per candidate. The same person can apply multiple times.

## 5. Statistical Summary

Understanding the distribution of our numeric columns.

In [ ]:
df.describe()

### Key insights:
- **YOE** (Years of Experience): ranges from 0 to 30, mean ~15 years
- **Code Challenge Score**: ranges 0-10, mean ~5.0 (uniformly distributed)
- **Technical Interview Score**: ranges 0-10, mean ~5.0 (uniformly distributed)
- Scores are evenly spread, which makes sense for randomly generated data

## 6. Categorical Columns Exploration

### 6.1 Seniority Distribution

In [ ]:
seniority_counts = df['Seniority'].value_counts()
print(seniority_counts)

plt.figure(figsize=(10, 5))
ax = seniority_counts.plot(kind='bar', color='#2ecc71', edgecolor='black')
plt.title('Candidates by Seniority Level', fontsize=14, fontweight='bold')
plt.xlabel('Seniority')
plt.ylabel('Count')
plt.xticks(rotation=45)

# Add count labels on bars
for i, v in enumerate(seniority_counts):
    ax.text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../diagrams/eda_seniority_distribution.png', dpi=150)
plt.show()

### 6.2 Technology Distribution

In [ ]:
tech_counts = df['Technology'].value_counts()
print(tech_counts)

plt.figure(figsize=(12, 6))
ax = tech_counts.plot(kind='barh', color='#3498db', edgecolor='black')
plt.title('Candidates by Technology', fontsize=14, fontweight='bold')
plt.xlabel('Count')
plt.ylabel('Technology')

plt.tight_layout()
plt.savefig('../diagrams/eda_technology_distribution.png', dpi=150)
plt.show()

### 6.3 Top 15 Countries

In [ ]:
country_counts = df['Country'].value_counts().head(15)

plt.figure(figsize=(12, 5))
ax = country_counts.plot(kind='bar', color='#e74c3c', edgecolor='black')
plt.title('Top 15 Countries by Number of Applications', fontsize=14, fontweight='bold')
plt.xlabel('Country')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')

for i, v in enumerate(country_counts):
    ax.text(i, v + 2, str(v), ha='center', fontweight='bold', fontsize=8)

plt.tight_layout()
plt.savefig('../diagrams/eda_top_countries.png', dpi=150)
plt.show()

## 7. Score Distributions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Code Challenge Score
axes[0].hist(df['Code Challenge Score'], bins=11, color='#9b59b6', edgecolor='black', alpha=0.8)
axes[0].set_title('Code Challenge Score Distribution', fontweight='bold')
axes[0].set_xlabel('Score')
axes[0].set_ylabel('Count')

# Technical Interview Score
axes[1].hist(df['Technical Interview Score'], bins=11, color='#f39c12', edgecolor='black', alpha=0.8)
axes[1].set_title('Technical Interview Score Distribution', fontweight='bold')
axes[1].set_xlabel('Score')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.savefig('../diagrams/eda_score_distributions.png', dpi=150)
plt.show()

## 8. Years of Experience Distribution

In [ ]:
plt.figure(figsize=(12, 5))
plt.hist(df['YOE'], bins=31, color='#1abc9c', edgecolor='black', alpha=0.8)
plt.title('Years of Experience (YOE) Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Years of Experience')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('../diagrams/eda_yoe_distribution.png', dpi=150)
plt.show()

## 9. Application Date Timeline

In [ ]:
df['Application Date'] = pd.to_datetime(df['Application Date'])

apps_by_month = df.groupby(df['Application Date'].dt.to_period('M')).size()
apps_by_month.index = apps_by_month.index.astype(str)

plt.figure(figsize=(14, 5))
plt.plot(range(len(apps_by_month)), apps_by_month.values, color='#2c3e50', linewidth=1.5)
plt.title('Applications Over Time (Monthly)', fontsize=14, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Applications')

# Show every 6th label to avoid clutter
tick_positions = range(0, len(apps_by_month), 6)
tick_labels = [apps_by_month.index[i] for i in tick_positions]
plt.xticks(list(tick_positions), tick_labels, rotation=45)

plt.tight_layout()
plt.savefig('../diagrams/eda_applications_timeline.png', dpi=150)
plt.show()

## 10. Hiring Rule Preview

**Business Rule:** A candidate is considered HIRED if:
- `Code Challenge Score >= 7` **AND** `Technical Interview Score >= 7`

Let's preview the impact of this rule on the dataset.

In [ ]:
hired_mask = (df['Code Challenge Score'] >= 7) & (df['Technical Interview Score'] >= 7)
hired_count = hired_mask.sum()
rejected_count = len(df) - hired_count

print(f'Total candidates:  {len(df):,}')
print(f'HIRED:             {hired_count:,} ({hired_count/len(df)*100:.1f}%)')
print(f'REJECTED:          {rejected_count:,} ({rejected_count/len(df)*100:.1f}%)')

# Pie chart
plt.figure(figsize=(7, 7))
plt.pie([hired_count, rejected_count],
        labels=['Hired', 'Rejected'],
        colors=['#2ecc71', '#e74c3c'],
        autopct='%1.1f%%',
        startangle=90,
        textprops={'fontsize': 14, 'fontweight': 'bold'})
plt.title('Hiring Outcome Preview', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../diagrams/eda_hiring_preview.png', dpi=150)
plt.show()

## 11. Score Correlation Heatmap

In [ ]:
numeric_cols = ['YOE', 'Code Challenge Score', 'Technical Interview Score']
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.3f',
            square=True, linewidths=1)
plt.title('Correlation Matrix - Numeric Columns', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../diagrams/eda_correlation_matrix.png', dpi=150)
plt.show()

## 12. EDA Summary & Key Findings

| Aspect | Finding |
|---|---|
| **Rows** | 50,000 candidate applications |
| **Columns** | 10 fields |
| **Null Values** | 0 — dataset is complete |
| **Duplicate Rows** | 0 — no full duplicates |
| **Duplicate Emails** | 167 — same candidates applied multiple times |
| **Date Range** | 2018-01-01 to 2022-07-04 |
| **Countries** | 244 unique countries |
| **Technologies** | 24 unique technologies |
| **Seniority Levels** | 7 levels (evenly distributed) |
| **Hiring Rate** | ~13.4% (6,698 hired out of 50,000) |
| **Score Distribution** | Uniform (0-10), mean ~5 |
| **YOE Range** | 0-30 years, mean ~15 |

### Decisions for ETL:
1. **Grain:** One row per application (not per candidate)
2. **Duplicate emails** are kept — multiple applications are valid
3. **Application Date** needs conversion from string to date type
4. **Hiring status** will be derived using the business rule in Transform phase
5. **No null handling needed** — dataset is complete